# Convolutional AutoEncoder for Denoising BSDS500 dataset

In [0]:
# Google Drive connectin
from google.colab import drive
drive.mount('/content/drive')

## Backend and Import

In [0]:
# Import Dependencies
import keras
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
import glob
import cv2

In [0]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
kerasBKED = os.environ["KERAS_BACKEND"] 
print(kerasBKED)

## Load BSDS500 dataset

In [0]:
# Download BSDS500 Dataset
!wget http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz

In [0]:
# Download testing image that will be used to compare.
!wget https://paragonexpeditions.com/wp-content/uploads/2017/04/farming-with-local-quechua-community-sacred-valley-of-the-incas.jpg

In [0]:
# Remove extra folder
!rm -r sample_data

In [0]:
#unpack.
!tar -xvf 'BSR_bsds500.tgz'

In [0]:
# setting of Parameters
batch_size = 16
epochs = 100
saveDir = "drive/My Drive/Colab Notebooks/ANN_Result/"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

Reading Dataset

In [0]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

# Read Training Data from filepath
files = glob.glob("BSR/BSDS500/data/images/train/*.jpg")
trainData = []
# Loop over files in filepath and add them to list.
for f1 in files:
    # Read Image
    img = cv2.imread(f1)
    # Resize Image
    img = cv2.resize(img, (400,400))
    # Extract BGR content 
    b,g,r = cv2.split(img)
    # Merge into an RGB image using the extracted content 
    img = cv2.merge([r,g,b]) 
    # Add image to List
    trainData.append(img)
    #Image Augmentation
    imgArr = img_to_array(img)
    samples = expand_dims(imgArr, 0)
    datagen = ImageDataGenerator(horizontal_flip=True)
    it = datagen.flow(samples, batch_size=1)
    # Generate a number of Images
    for i in range(3):
      batch = it.next()
      image = batch[0].astype('uint8')
      trainData.append(image)


# Read Testing Data from filepath
files = glob.glob("BSR/BSDS500/data/images/test/*.jpg")
testData = []
# Loop over files in filepath and add them to list.
for f1 in files:
    # Read Image
    img = cv2.imread(f1)
    # Resize Image
    img = cv2.resize(img, (400,400))
    # Extract BGR content 
    b,g,r = cv2.split(img)
    # Merge into an RGB image using the extracted content  
    img = cv2.merge([r,g,b]) 
    # Add image to List
    testData.append(img)
    # Image Augmentation
    imgArr = img_to_array(img)
    samples = expand_dims(imgArr, 0)
    datagen = ImageDataGenerator(horizontal_flip=True)
    it = datagen.flow(samples, batch_size=1)
    # Generate a Number of Images
    for i in range(3):
      batch = it.next()
      image = batch[0].astype('uint8')
      testData.append(image)

files = glob.glob("BSR/BSDS500/data/images/val/*.jpg")
# Loop over files in filepath and add them to list.
for f1 in files:
    # Read Image
    img = cv2.imread(f1)
    # Resize Image
    img = cv2.resize(img, (400,400))
    # Extract BGR content 
    b,g,r = cv2.split(img)
    # Merge into an RGB image using the extracted content  
    img = cv2.merge([r,g,b]) 
    # Add image to List
    trainData.append(img)

    imgArr = img_to_array(img)
    samples = expand_dims(imgArr, 0)
    datagen = ImageDataGenerator(horizontal_flip=True)
    it = datagen.flow(samples, batch_size=1)
    for i in range(9):
      batch = it.next()
      image = batch[0].astype('uint8')
      trainData.append(image)
# Read a Test Image used for displaying.
img = cv2.imread("farming-with-local-quechua-community-sacred-valley-of-the-incas.jpg")
img = cv2.resize(img, (400,400))
b,g,r = cv2.split(img)
img = cv2.merge([r,g,b])
testData.append(img)

# Change to Numpy Array   
trainData = np.array(trainData)
testData = np.array(testData)

x_train = trainData
x_test = testData

# Shuffle Dataset after Augmentation
np.random.shuffle(x_train)
np.random.shuffle(x_test[:-2])

#Display Shape
print("x_train.shape: " + str(x_train.shape))
print("x_test.shape: " + str(x_test.shape))

In [0]:
# Display Sample from Train Set
import matplotlib.pyplot as plt
plt.imshow(x_train[0])
plt.show()

In [0]:
# normalize data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [0]:
# divide x_test into validation and test
x_val = x_test[:400]
x_test = x_test[400:]

In [0]:
# Data Shapes for Integrity check
print("validation data: {0} \ntest data: {1}".format(x_val.shape, x_test.shape))

In [0]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0].reshape(400, 400, 3))
plt.show()

## add noise to data

In [0]:
# Adding of noise to the np array of images using additive noise factoring and np normal
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
x_val_noisy = x_val + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_val.shape) 


x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
x_val_noisy = np.clip(x_val_noisy, 0., 1.)


## show noisy images

In [0]:
# definition to show original image and reconstructed image
def showOrigDec(orig, noise, num=3):
    import matplotlib.pyplot as plt
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[i].reshape(400, 400, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        # display original
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(noise[i].reshape(400, 400, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [0]:
showOrigDec(x_train, x_train_noisy)

## Convolutional AutoEncoder for denoising

In [0]:
input_img = Input(shape=(400, 400, 3))
#ENCODE
x = Conv2D(100, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(50, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
#CODE Representation
encoded = MaxPooling2D((2, 2), padding='same')(x)
#DECODE
x = Conv2D(50, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(100, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)

In [0]:
from keras import metrics
# Start Model and set Optimizer, loss function and metrics.
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', metrics.mae, metrics.mse])

## Train AutoEncoder

In [0]:
%load_ext tensorboard

In [0]:
# Clear any logs from previous runs
# !rm -rf ./my_log_dir/
#import os
#os.rmdir("./my_log_dir/")
# shutil.rmtree('./my_log_dir/')

In [0]:
# load pretrained weights
# model.load_weights(saveDir + 'AutoEncoder.hdf5')

In [0]:
# Callbakc functions that will be called on epoch.
from keras.callbacks import TensorBoard
es_cb = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
chkpt = saveDir + 'AutoEncoder.hdf5'
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
tfBoard = TensorBoard(
        log_dir="my_log_dir",
        histogram_freq=1,

    )
reduceLR = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.01,
        patience=10,
    )

In [0]:
#Start Fitting the NN using the settings provided above.
history = model.fit(x_train_noisy, x_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val_noisy, x_val),
                    callbacks=[es_cb, cp_cb, tfBoard, reduceLR],
                    shuffle=True)

In [0]:
# Display Tensorboard.
#!kill 507 
%tensorboard --logdir=my_log_dir --host localhost --port=8005

In [0]:
from tensorflow.keras.utils import plot_model
# Display Model Structure
plot_model(model, to_file='model.png')

## Evaluate with test dataset

In [0]:
score = model.evaluate(x_test_noisy, x_test, verbose=1)
print(score)

In [0]:
# Test Image Arrays.
c10test = model.predict(x_test_noisy)
c10val = model.predict(x_val_noisy)

In [0]:
print("test score: {0}\nval score: {1}".format(np.average(c10test), np.average(c10val)))

In [0]:
# definition to show original image and reconstructed image
def showOrigDec(orig, noise, denoise, num=10):
    import matplotlib.pyplot as plt
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # display original
        ax = plt.subplot(3, n, i+1)
        plt.imshow(orig[i].reshape(400, 400, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display noisy image
        ax = plt.subplot(3, n, i +1 + n)
        plt.imshow(noise[i].reshape(400, 400, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        # display denoised image
        ax = plt.subplot(3, n, i +1 + n + n)
        plt.imshow(denoise[i].reshape(400, 400, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [0]:
# Display a number of Images (default=10)
showOrigDec(x_test, x_test_noisy, c10test)

In [0]:
c10test = model.predict(x_test_noisy)

In [0]:
import matplotlib.pyplot as plt
imgNum = -1
f, axarr = plt.subplots(1, 3, figsize=(26, 6))
axarr[0].imshow(x_test[imgNum], aspect='auto')
axarr[0].get_xaxis().set_visible(False)
axarr[0].get_yaxis().set_visible(False)
axarr[1].imshow(x_test_noisy[imgNum], aspect='auto')
axarr[1].get_xaxis().set_visible(False)
axarr[1].get_yaxis().set_visible(False)
axarr[2].imshow(c10test[imgNum], aspect='auto')
axarr[2].get_xaxis().set_visible(False)
axarr[2].get_yaxis().set_visible(False)
